# Outlier The Year in Fabrics &mdash; 2024, companion

Accompaniment to Abe's [2024 Year in Fabrics](https://www.reddit.com/r/Outlier/comments/1hssnan/2024_year_in_fabrics/)

In [1]:
import pandas as pd
import numpy as np
import scipy
import os
import os.path
from IPython.display import display, HTML
import utils
import plots
import altair as alt

year = 2024
df_all = utils.outlier_df('../../outlier-data.csv', year)
# df_all = utils.outlier_df('../../../outlier/public/outlier-data.csv', year)

/Users/ciyer/Documents/projects/outlier/outlier/public/articles/2024-review/utils.py:68: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(price * bpl / factor)


In [2]:
# rel = df_all.iloc[0]['Release'].to_pydatetime()
# <i>(Provisional, containing drops up to {rel.strftime("%b %d, %Y")})</i></p>
text = f"""<p>A catalog of all drops from {year}, grouped by fabric, ordered by the number of drops in that fabric.<br />
"""
display(HTML(text))

In [3]:
# Filter to Clothes
def type_to_kind(drop_type):
    if drop_type == "T-Shirt" or drop_type == "Shirt" or drop_type == "Tank Top":
        return "Top"
    if drop_type == "Pants" or drop_type == "Shorts":
        return "Bottom"
    return "Layer"


def row_to_subkind(row):
    drop_type = row['Type']
    if drop_type == "T-Shirt" or drop_type == "Shirt":
        pass
    elif drop_type == "Pants" or drop_type == "Shorts":
        pass
    elif drop_type == "Tank Top":
        drop_type = 'T-Shirt'
    else:
        drop_type = "Layer"
    exp = "Exp" if row['Exp'] else "Regular"
    return "{}/{}".format(drop_type, exp)


def object_type_to_kind(drop_type):
    if drop_type == "Backpack" or drop_type == "Bag" or drop_type == "Dopp Kit" \
       or drop_type == "Dufflebag" or drop_type == "Pouch":
        return "Bag"
    if drop_type == "Balaclava" or drop_type == "Bandana" or drop_type == "Scarf":
        return "Scarf/Bandana"
    if drop_type == "Cap" or drop_type == "Hat" or drop_type == "Hood":
        return "Headgear"
    if drop_type == "Bathrobe" or drop_type == "Robe" or drop_type == "Towel" \
       or drop_type == "Outerwear" or drop_type == "Keychain" or drop_type == "Socks":
        return "Misc"
    if drop_type == "Mask" or drop_type == "Belt":
        return drop_type
    return "Misc"

# Define some reused frames
all_products_df = utils.drops_to_products(df_all, ['Year', 'Product'], year)
df_all['ColorCount'] = df_all['Colors'].str.split(',').dropna().apply(lambda d: len(d))

df_clothes = pd.DataFrame(df_all.loc[df_all['Category'] == 'Clothes', :])
df_clothes['Type'] = df_clothes['Type'].replace({'Sweatshirt':'Sweater'})
df_clothes["Kind"] = df_clothes['Type'].apply(type_to_kind)
df_clothes["Subkind"] = df_clothes.apply(row_to_subkind, axis=1)
clothes_products_df = utils.drops_to_products(df_clothes, ['Year', 'Product'], year)

df_objects = pd.DataFrame(df_all.loc[df_all['Category'] == 'Objects', :])
df_objects["Kind"] = df_objects['Type'].apply(object_type_to_kind)
objects_products_df = utils.drops_to_products(df_objects, ['Year', 'Product'], year)


kind_sort_order = ['Bottom', 'Top', 'Layer']
object_kind_sort_order = ['Bag', 'Mask', 'Scarf/Bandana', 'Headgear', 'Belt', 'Other']
year_sort = list(range(year, 2012, -1))

In [4]:
df_year = df_all[df_all['Release'].dt.year == year]

In [5]:
from plots import product_summary_html

def fabric_summary_display(df_all, fabric, number_of_drops):
    # Order by number of drops, not alphabetically
    # products = sorted(products)
    products = df_all[df_all['Fabric'] == fabric].groupby('Product').count()['Type'].sort_values(ascending=False).index
    if len(products) < 1:
        return
    htmls = [product_summary_html(df_all, p) for p in products]
    fragments = [f"<div class='product'>{h.data}</div>" for h in htmls]
    fabric_count_str = f"<span style='color: #888; font-size: 0.8em;'> ({number_of_drops} drops)</span>" if number_of_drops > 1 else ""
    row = f"<h3>{fabric}{fabric_count_str}</h3><div class='fabric-products' style='display: flex'>{' '.join(fragments)}</div>"
    display(HTML(row))

In [6]:
# Show in order of most-used to least-used fabric
# fabrics = sorted(df_2021['Fabric'].unique())
tdf = df_year.groupby('Fabric').count()['Product'].reset_index()
tdf = tdf.sort_values(['Product', 'Fabric'], ascending=[False, True])
fabrics = tdf['Fabric']
for f in tdf.iterrows():
    fabric_summary_display(df_year, f[1]['Fabric'], f[1]['Product'])

# TODO: do not need to see last drop, would be interesting to see first drop

,Value
last drop,2024
price,198
number of drops,2
fabric,Injected Linen
number of colors,6
colors,"RD Black, RD Concrete, RD Darkness, RD Dry Tan, RD Medjool, RD Olive Territory"
,Value
last drop,2024
price,333
number of drops,1


,Value
last drop,2024
price,395
number of drops,2
fabric,Hard/Co Merino
number of colors,5
colors,"Anthracite, Black, Coraleyes, Dark Navy, Sage"
,Value
last drop,2024
price,495
number of drops,1


,Value
last drop,2024
price,$198 — 225
number of drops,2
fabric,Bombtwill
number of colors,4
colors,"Black, Dry Tan, DryTan, Grayzone"
,Value
last drop,2024
price,160
number of drops,1


,Value
last drop,2024
price,175
number of drops,3
fabric,F. Cloth
number of colors,11
colors,"Black, Dark Navy, DarkOlive, DeepInk, Dry Tan, Dust Olive, Medjool, Olive Territory, Sagebrush, Sandstorm, Tannic"
,Value
last drop,2024
price,175
number of drops,2


,Value
last drop,2024
price,$225 — 248
number of drops,4
fabric,Acottontwill
number of colors,13
colors,"Articsky, Bluecoral, Blueyellow, Dragonsblood, Dropcloth, Firecracker, Iridescent, Navysecret, Notdenim, Notdenimfade, Olivecoral, Seagate, Shadowforrest"
,Value
last drop,2024
price,175
number of drops,1


...

,Value
last drop,2024
price,555
number of drops,1
fabric,Paracord 550 Type III
number of colors,1
colors,11Custom Paracords (+Black)


,Value
last drop,2024
price,525
number of drops,1
fabric,"Strongtwill, Alphacore"
number of colors,4
colors,"Charcoal, Concrete, Dark Olive, Winedark"


,Value
last drop,2024
price,175
number of drops,1
fabric,Supermarine
number of colors,1
colors,Blacknavy


,Value
last drop,2024
price,475
number of drops,1
fabric,Trackwool
number of colors,1
colors,Navy


,Value
last drop,2024
price,1295
number of drops,1
fabric,adown(ventile)
number of colors,1
colors,Black
